# Coursera Capstone Final Project: Analysing Detroit Neighborhoods

Neighborhood map shape files were taken from https://data.detroitmi.gov/ and the shape files were transformed into Longitude and Latitude using https://mygeodata.cloud/converter/shp-to-latlong

In [1]:
import json # library to handle JSON files

import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans, DBSCAN

In [2]:
!pip install folium
import folium

     |████████████████████████████████| 94 kB 7.4 MB/s  eta 0:00:01


In [3]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_28756a0d5ecb44599d434ba14696fd67 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_28756a0d5ecb44599d434ba14696fd67.get_object(Bucket='courseraassignment-donotdelete-pr-kf68n2kx7h1rqu',Key='Detroit_Neighborhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,OBJECTID,number,name,X,Y,council_di,Website,SHAPE_Leng,SHAPE_Area
0,1,103,West End,-82.997266,42.383644,4,https://theneighborhoods.org/neighborhoods/wes...,0.098727,0.000322
1,2,105,Chandler Park,-82.974079,42.396291,4,https://theneighborhoods.org/neighborhoods/cha...,0.063939,0.000220
2,3,100,Riverbend,-82.953467,42.379379,4,https://theneighborhoods.org/neighborhoods/riv...,0.074513,0.000317
3,4,101,Jefferson Chalmers,-82.945100,42.364698,4,https://theneighborhoods.org/neighborhoods/jef...,0.083501,0.000399
4,5,29,Riverdale,-83.267268,42.410826,1,https://theneighborhoods.org/neighborhoods/riv...,0.054555,0.000143


In [4]:
df_data_1.shape

(208, 9)

In [5]:
df_data_1.rename(columns = {'X': 'longitude', 'Y': 'latitude', 'name': 'Neighborhood'}, inplace = True )
df_data_1.head()

,OBJECTID,number,Neighborhood,longitude,latitude,council_di,Website,SHAPE_Leng,SHAPE_Area
0,1,103,West End,-82.997266,42.383644,4,https://theneighborhoods.org/neighborhoods/wes...,0.098727,0.000322
1,2,105,Chandler Park,-82.974079,42.396291,4,https://theneighborhoods.org/neighborhoods/cha...,0.063939,0.000220
2,3,100,Riverbend,-82.953467,42.379379,4,https://theneighborhoods.org/neighborhoods/riv...,0.074513,0.000317
3,4,101,Jefferson Chalmers,-82.945100,42.364698,4,https://theneighborhoods.org/neighborhoods/jef...,0.083501,0.000399
4,5,29,Riverdale,-83.267268,42.410826,1,https://theneighborhoods.org/neighborhoods/riv...,0.054555,0.000143


In [6]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20210205' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

neighborhood_latitude = df_data_1.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df_data_1.loc[0, 'longitude'] # neighborhood longitude value

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=11RINKB11AJXLQDTEOBFLQJKEES4XR0LVE4Q3P13P0DV1RFT&client_secret=PWMWC0PS1XJZEBELDQ5HBIZVIMTCFGVRXNAEVB0DIIVS15XJ&v=20210205&ll=42.38364406,-82.99726649&radius=500&limit=100'

In [7]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '601f1dfc83076b1f2315b1c0'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Lower East Detroit',
  'headerFullLocation': 'Lower East Detroit, Detroit',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 42.388144064500004,
    'lng': -82.99118564541475},
   'sw': {'lat': 42.3791440555, 'lng': -83.00334733458526}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5317b3e511d2476e831493e7',
       'name': 'Cadillac & East Warren Avenues',
       'location': {'address': 'Cadillac Ave & E Warren Ave',
        'lat': 42.382339262355174,
        'lng': -82.996795177

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
Detroit_venues = getNearbyVenues(names=df_data_1['Neighborhood'],
                                   latitudes=df_data_1['latitude'],
                                   longitudes=df_data_1['longitude']
                                  )

West End
Chandler Park
Riverbend
Jefferson Chalmers
Riverdale
Morningside
Cornerstone Village
Regent Park
North Rosedale Park
Berg-Lahser
Five Points
The Eye
Old Redford
Grandmont
Grandmont #1
Rosedale Park
Minock Park
Westwood Park
Warrendale
Garden View
Warren Ave Community
Far West Detroit
Marina District
Arden Park
Joseph Berry Sub
Indian Village
Rivertown
West Village
Conner Creek Industrial
Belmont
Corktown
Boynton
Oakwood Heights
Blackstone Park
Delray
Claytown
Aviation Sub
Barton-McFarland
Medical Center
Bagley
Green Acres
State Fair
Palmer Woods
Sherwood Forest
University District
Islandview
Springwells
Franklin Park
Weatherby
Castle Rouge
Eliza Howell
Detroit Golf
Brightmoor
Chadsey Condon
Woodbridge
Greenfield
Oakman Blvd Community
Nardin Park
Petoskey-Otsego
Boston Edison
Campau/Banglatown
Conant Gardens
Krainz Woods
LaSalle College Park
Elmwood Park
McDougall-Hunt
North Corktown
Cultural Center
Carbon Works
Michigan-Martin
Pingree Park
Martin Park
East Village
Hubbard Farm

In [10]:
Detroit_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,West End,42.383644,-82.997266,Cadillac & East Warren Avenues,42.382339,-82.996795,Intersection
1,West End,42.383644,-82.997266,Bitcoin of America - Bitcoin ATM,42.382595,-82.997384,Currency Exchange
2,West End,42.383644,-82.997266,Jims Beer and Wine Store,42.384396,-82.991987,Wine Bar
3,Chandler Park,42.396291,-82.974079,Chandler Park Golf,42.397388,-82.974174,Golf Course
4,Chandler Park,42.396291,-82.974079,Chandler Park Water Park,42.394158,-82.976863,Beach


In [11]:
Detroit_venues.shape

(1133, 7)

In [12]:
pd.options.display.max_rows = 4000
venue_types = Detroit_venues['Venue Category'].unique()
print(venue_types) #array containing the names of all the unique venue types

['Intersection' 'Currency Exchange' 'Wine Bar' 'Golf Course' 'Beach'
 'Park' 'Tree' 'Construction & Landscaping' 'Bar' "Men's Store"
 'Arts & Crafts Store' 'Pet Store' 'Deli / Bodega' 'Discount Store'
 'Chinese Restaurant' 'Fried Chicken Joint' 'Burger Joint' 'Fishing Spot'
 'Home Service' 'Gym' 'Smoke Shop' 'Grocery Store' 'Recreation Center'
 'Coffee Shop' 'Bakery' 'Pizza Place' 'Liquor Store' 'Mobile Phone Shop'
 'Playground' 'Food' 'Southern / Soul Food Restaurant'
 'Indie Movie Theater' 'Dessert Shop' 'Diner' 'Fast Food Restaurant'
 'Other Repair Shop' 'Art Gallery' 'Cosmetics Shop' 'Movie Theater'
 'Pharmacy' 'Furniture / Home Store' 'Café' 'Baseball Field' 'Gay Bar'
 'Gas Station' 'Bank' 'Convenience Store' 'Food Truck'
 'Entertainment Service' 'Seafood Restaurant' 'Ice Cream Shop'
 'Harbor / Marina' 'Paper / Office Supplies Store' 'Historic Site'
 'Health Food Store' 'Boat or Ferry' 'Sporting Goods Shop'
 'New American Restaurant' 'French Restaurant' 'Athletics & Sports'
 'Gym 

In [13]:
Detroit_venues['Venue Category'].unique().shape #Number of unique venue types

(220,)

In [14]:
Detroit_venues.groupby(by = ['Neighborhood']).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arden Park,2,2,2,2,2,2
Aviation Sub,2,2,2,2,2,2
Bagley,4,4,4,4,4,4
Barton-McFarland,4,4,4,4,4,4
Belle Isle,1,1,1,1,1,1
Belmont,1,1,1,1,1,1
Berg-Lahser,1,1,1,1,1,1
Bethune Community,7,7,7,7,7,7
Blackstone Park,1,1,1,1,1,1


In [15]:
Venue_counts = Detroit_venues.groupby(by = ['Neighborhood']).count()
Venue_counts.reset_index(inplace = True)
Venue_counts = Venue_counts[['Neighborhood', 'Venue']] # get number of venues per neighborhood


df_data_1.drop(['OBJECTID', 'number'], axis = 1)
Venue_counts = df_data_1.merge(Venue_counts, how = 'left', on = 'Neighborhood')
Venue_counts.head()


,OBJECTID,number,Neighborhood,longitude,latitude,council_di,Website,SHAPE_Leng,SHAPE_Area,Venue
0,1,103,West End,-82.997266,42.383644,4,https://theneighborhoods.org/neighborhoods/wes...,0.098727,0.000322,3.0
1,2,105,Chandler Park,-82.974079,42.396291,4,https://theneighborhoods.org/neighborhoods/cha...,0.063939,0.000220,3.0
2,3,100,Riverbend,-82.953467,42.379379,4,https://theneighborhoods.org/neighborhoods/riv...,0.074513,0.000317,3.0
3,4,101,Jefferson Chalmers,-82.945100,42.364698,4,https://theneighborhoods.org/neighborhoods/jef...,0.083501,0.000399,1.0
4,5,29,Riverdale,-83.267268,42.410826,1,https://theneighborhoods.org/neighborhoods/riv...,0.054555,0.000143,1.0


In [16]:
Venue_counts['Venue'].fillna(0, inplace = True) # replace neighborhoods that returned no venues with zeros
Venue_counts['Venue'].astype(int)

0       3
1       3
2       3
3       1
4       1
5       5
6       8
7       2
8       6
9       1
10      4
11      2
12      9
13      1
14      4
15      5
16      6
17      2
18      6
19      0
20      7
21      2
22      8
23      2
24      8
25      6
26     33
27      9
28      3
29      1
30     26
31      3
32      2
33      1
34      1
35     12
36      2
37      4
38     11
39      4
40      1
41     11
42      1
43      8
44      4
45      4
46     11
47      0
48      4
49      0
50      0
51      5
52      0
53      1
54     10
55      2
56      6
57      2
58      6
59      2
60      4
61      4
62      5
63      4
64      1
65      4
66      4
67     20
68      2
69      6
70      4
71      2
72      3
73     10
74      5
75      6
76      2
77      6
78      7
79      2
80     17
81      6
82      6
83      2
84      9
85      2
86      4
87      2
88     20
89     48
90     18
91      1
92      0
93      7
94      1
95      4
96      4
97     19
98      4
99      4


In [17]:
#Use K means to segment neighborhoods
k_means = KMeans(init = 'k-means++', n_clusters = 5)
X = np.asarray(Venue_counts[['Venue']])
k_means.fit(X)
k_means_labels = k_means.labels_

In [18]:
Venue_counts['Cluster Labels'] = k_means_labels
Venue_counts.head()

,OBJECTID,number,Neighborhood,longitude,latitude,council_di,Website,SHAPE_Leng,SHAPE_Area,Venue,Cluster Labels
0,1,103,West End,-82.997266,42.383644,4,https://theneighborhoods.org/neighborhoods/wes...,0.098727,0.000322,3.0,4
1,2,105,Chandler Park,-82.974079,42.396291,4,https://theneighborhoods.org/neighborhoods/cha...,0.063939,0.000220,3.0,4
2,3,100,Riverbend,-82.953467,42.379379,4,https://theneighborhoods.org/neighborhoods/riv...,0.074513,0.000317,3.0,4
3,4,101,Jefferson Chalmers,-82.945100,42.364698,4,https://theneighborhoods.org/neighborhoods/jef...,0.083501,0.000399,1.0,4
4,5,29,Riverdale,-83.267268,42.410826,1,https://theneighborhoods.org/neighborhoods/riv...,0.054555,0.000143,1.0,4


In [19]:
map_clusters = folium.Map(location=[42.3314, -83.0458], zoom_start=11)


x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Venue_counts['latitude'], Venue_counts['longitude'], Venue_counts['Neighborhood'], Venue_counts['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [20]:
Venue_counts.groupby(by = ['Cluster Labels']).mean()

,OBJECTID,number,longitude,latitude,council_di,SHAPE_Leng,SHAPE_Area,Venue
Cluster Labels,,,,,,,,
0,96.130435,86.304348,-83.118444,42.387788,3.347826,0.048381,0.000143,9.608696
1,103.507042,104.887324,-83.104352,42.393389,4.000000,0.056050,0.000163,4.957746
2,140.000000,143.666667,-83.045374,42.339081,5.333333,0.053172,0.000155,47.000000
3,98.000000,151.272727,-83.065410,42.345009,5.454545,0.056103,0.000156,23.090909
4,106.780000,102.090000,-83.117987,42.391909,3.920000,0.063890,0.000223,1.670000


In [21]:
db = DBSCAN(eps = 0.2, min_samples = 17).fit(X)
Venue_counts['DB Labels'] = db.labels_
Venue_counts.head()

,OBJECTID,number,Neighborhood,longitude,latitude,council_di,Website,SHAPE_Leng,SHAPE_Area,Venue,Cluster Labels,DB Labels
0,1,103,West End,-82.997266,42.383644,4,https://theneighborhoods.org/neighborhoods/wes...,0.098727,0.000322,3.0,4,0
1,2,105,Chandler Park,-82.974079,42.396291,4,https://theneighborhoods.org/neighborhoods/cha...,0.063939,0.000220,3.0,4,0
2,3,100,Riverbend,-82.953467,42.379379,4,https://theneighborhoods.org/neighborhoods/riv...,0.074513,0.000317,3.0,4,0
3,4,101,Jefferson Chalmers,-82.945100,42.364698,4,https://theneighborhoods.org/neighborhoods/jef...,0.083501,0.000399,1.0,4,1
4,5,29,Riverdale,-83.267268,42.410826,1,https://theneighborhoods.org/neighborhoods/riv...,0.054555,0.000143,1.0,4,1


In [22]:
Venue_counts['DB Labels'].unique()

array([ 0,  1,  2, -1,  3,  4])

__After testing multiple parameters for DBSCAN on the venue count data, it becomes clear that the min samples must be set very high in order to reduce the number of unique label. Given how high the min sample must be set, the segmentation stops producing meaningful segments.__

In [23]:
map_clusters_db = folium.Map(location=[42.3314, -83.0458], zoom_start=11)


x = np.arange(6)
ys = [i + x + (i*x)**2 for i in range(6)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Venue_counts['latitude'], Venue_counts['longitude'], Venue_counts['Neighborhood'], Venue_counts['DB Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_db)
       
map_clusters_db

In [24]:
DV1 = Detroit_venues.groupby(by = ['Neighborhood', 'Venue Category']).count()
DV1.reset_index(inplace = True)
DV1 = DV1[['Neighborhood', 'Venue Category', 'Venue']]
DV_pivot = DV1.pivot(index = 'Neighborhood', columns = 'Venue Category')
DV_pivot.fillna(0, inplace = True)
DV_pivot.head()

Venue                                                     \
Venue Category     ATM Accessories Store American Restaurant Antique Shop   
Neighborhood                                                                
Arden Park         0.0               0.0                 0.0          0.0   
Aviation Sub       0.0               0.0                 0.0          0.0   
Bagley             0.0               0.0                 0.0          0.0   
Barton-McFarland   0.0               0.0                 1.0          0.0   
Belle Isle         0.0               0.0                 0.0          0.0   

                                                                              \
Venue Category   Art Gallery Art Museum Arts & Crafts Store Asian Restaurant   
Neighborhood                                                                   
Arden Park               0.0        0.0                 0.0              0.0   
Aviation Sub             0.0        0.0                 0.0              0.0   
Bagley                   0.0        0.0                 0.0              0.0   
Barton-McFarland         0.0        0.0                 0.0              0.0   
Belle Isle               0.0        0.0                 0.0              0.0   

                                                 ...                       \
Venue Category   Athletics & Sports Auto Garage  ... Tree Udon Restaurant   
Neighborhood                                     ...                        
Arden Park                      0.0         0.0  ...  0.0             0.0   
Aviation Sub                    0.0         0.0  ...  0.0             0.0   
Bagley                          0.0         0.0  ...  0.0             0.0   
Barton-McFarland                0.0         0.0  ...  0.0             0.0   
Belle Isle                      0.0         0.0  ...  0.0             0.0   

                                                                      \
Venue Category   Vegetarian / Vegan Restaurant Vietnamese Restaurant   
Neighborhood                                                           
Arden Park                                 0.0                   0.0   
Aviation Sub                               0.0                   0.0   
Bagley                                     0.0                   0.0   
Barton-McFarland                           0.0                   0.0   
Belle Isle                                 0.0                   0.0   

                                                                          \
Venue Category   Whisky Bar Wine Bar Wine Shop Wings Joint Women's Store   
Neighborhood                                                               
Arden Park              0.0      0.0       0.0         0.0           0.0   
Aviation Sub            0.0      0.0       0.0         0.0           0.0   
Bagley                  0.0      0.0       0.0         0.0           0.0   
Barton-McFarland        0.0      0.0       0.0         0.0           0.0   
Belle Isle              0.0      0.0       0.0         0.0           0.0   

                              
Venue Category   Yoga Studio  
Neighborhood                  
Arden Park               0.0  
Aviation Sub             0.0  
Bagley                   0.0  
Barton-McFarland         0.0  
Belle Isle               0.0  

[5 rows x 220 columns]

In [25]:
DV_pivot.columns = DV_pivot.columns.to_frame().astype(str).apply(','.join, axis=1) #flatten multi-index in columns to perform merge
DV_pivot.columns

Index(['Venue,ATM', 'Venue,Accessories Store', 'Venue,American Restaurant',
       'Venue,Antique Shop', 'Venue,Art Gallery', 'Venue,Art Museum',
       'Venue,Arts & Crafts Store', 'Venue,Asian Restaurant',
       'Venue,Athletics & Sports', 'Venue,Auto Garage',
       ...
       'Venue,Tree', 'Venue,Udon Restaurant',
       'Venue,Vegetarian / Vegan Restaurant', 'Venue,Vietnamese Restaurant',
       'Venue,Whisky Bar', 'Venue,Wine Bar', 'Venue,Wine Shop',
       'Venue,Wings Joint', 'Venue,Women's Store', 'Venue,Yoga Studio'],
      dtype='object', length=220)

In [26]:
DV_pivot.shape

(194, 220)

In [27]:
Venue_counts.shape

(208, 12)

In [28]:
k_means = KMeans(init = 'k-means++', n_clusters = 5)

k_means.fit(DV_pivot)
k_means_labels_DVP = k_means.labels_
k_means_labels_DVP[1:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [42]:
DV_pivot['Pivot Cluster Labels'] = k_means_labels_DVP
DV_pivot.head()

,"Venue,ATM","Venue,Accessories Store","Venue,American Restaurant","Venue,Antique Shop","Venue,Art Gallery","Venue,Art Museum","Venue,Arts & Crafts Store","Venue,Asian Restaurant","Venue,Athletics & Sports","Venue,Auto Garage",...,"Venue,Vegetarian / Vegan Restaurant","Venue,Vietnamese Restaurant","Venue,Whisky Bar","Venue,Wine Bar","Venue,Wine Shop","Venue,Wings Joint","Venue,Women's Store","Venue,Yoga Studio",DB Pivot Labels,Pivot Cluster Labels
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Arden Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,1
Aviation Sub,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,1
Bagley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,1
Barton-McFarland,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,1
Belle Isle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [43]:
DV_pivot.groupby(by = 'Pivot Cluster Labels').mean()

,"Venue,ATM","Venue,Accessories Store","Venue,American Restaurant","Venue,Antique Shop","Venue,Art Gallery","Venue,Art Museum","Venue,Arts & Crafts Store","Venue,Asian Restaurant","Venue,Athletics & Sports","Venue,Auto Garage",...,"Venue,Udon Restaurant","Venue,Vegetarian / Vegan Restaurant","Venue,Vietnamese Restaurant","Venue,Whisky Bar","Venue,Wine Bar","Venue,Wine Shop","Venue,Wings Joint","Venue,Women's Store","Venue,Yoga Studio",DB Pivot Labels
Pivot Cluster Labels,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,0.000000,0.000000,-1.000000
1,0.022857,0.005714,0.108571,0.005714,0.057143,0.011429,0.011429,0.0,0.022857,0.011429,...,0.000000,0.011429,0.000000,0.000000,0.051429,0.0,0.017143,0.022857,0.005714,-0.828571
2,0.000000,0.000000,0.800000,0.066667,0.266667,0.000000,0.000000,0.0,0.000000,0.000000,...,0.066667,0.066667,0.066667,0.066667,0.133333,0.0,0.066667,0.000000,0.000000,-1.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.500000,0.000000,0.000000,-1.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,-1.000000


In [88]:
DVP = DV_pivot.groupby(by = 'Pivot Cluster Labels').mean()

In [93]:
DVP = DVP.transpose()
DVP.sort_values(by = 0, ascending = False).head()



Pivot Cluster Labels,0,1,2,3,4
"Venue,Farmers Market",6.0,0.000000,0.066667,0.0,0.0
"Venue,Coffee Shop",3.0,0.085714,0.733333,0.5,7.0
"Venue,Food",2.0,0.062857,0.066667,0.0,0.0
"Venue,Clothing Store",2.0,0.045714,0.200000,1.0,0.0
"Venue,Deli / Bodega",2.0,0.034286,0.266667,0.0,0.0


In [94]:
DVP.sort_values(by = 1, ascending = False).head()

Pivot Cluster Labels,0,1,2,3,4
"Venue,Liquor Store",0.0,0.182857,0.066667,0.0,0.0
"Venue,Intersection",0.0,0.131429,0.200000,0.0,0.0
"Venue,Discount Store",0.0,0.125714,0.266667,0.0,0.0
"Venue,Bar",1.0,0.120000,0.866667,0.0,1.0
"Venue,Pizza Place",1.0,0.108571,0.866667,1.0,1.0


In [95]:
DVP.sort_values(by = 2, ascending = False).head()

Pivot Cluster Labels,0,1,2,3,4
"Venue,Pizza Place",1.0,0.108571,0.866667,1.0,1.0
"Venue,Bar",1.0,0.120000,0.866667,0.0,1.0
"Venue,American Restaurant",0.0,0.108571,0.800000,0.0,0.0
"Venue,Coffee Shop",3.0,0.085714,0.733333,0.5,7.0
"Venue,Mexican Restaurant",0.0,0.028571,0.733333,0.0,0.0


In [96]:
DVP.sort_values(by = 3, ascending = False).head()

Pivot Cluster Labels,0,1,2,3,4
"Venue,Plaza",0.0,0.000000,0.133333,1.5,0.0
"Venue,Chinese Restaurant",0.0,0.080000,0.600000,1.0,0.0
"Venue,Hotel",0.0,0.005714,0.133333,1.0,4.0
"Venue,Park",0.0,0.097143,0.200000,1.0,4.0
"Venue,Performing Arts Venue",0.0,0.011429,0.000000,1.0,0.0


In [97]:
DVP.sort_values(by = 4, ascending = False).head()

Pivot Cluster Labels,0,1,2,3,4
"Venue,Coffee Shop",3.0,0.085714,0.733333,0.5,7.0
"Venue,Lounge",0.0,0.022857,0.066667,0.5,5.0
"Venue,Park",0.0,0.097143,0.200000,1.0,4.0
"Venue,Hotel",0.0,0.005714,0.133333,1.0,4.0
"Venue,Restaurant",0.0,0.017143,0.400000,0.0,3.0


In [31]:
DV_pivot.reset_index(inplace = True)
DV1 = DV_pivot[['Neighborhood', 'Pivot Cluster Labels']]
DV1.reset_index(inplace = True, drop = True)

DV1.head()

,Neighborhood,Pivot Cluster Labels
0,Arden Park,1
1,Aviation Sub,1
2,Bagley,1
3,Barton-McFarland,1
4,Belle Isle,1


In [32]:
Neighborhoods_LL = Venue_counts[['Neighborhood', 'longitude', 'latitude']]
DV1 = Neighborhoods_LL.merge(DV1, on = ['Neighborhood'], how = 'outer')
DV1.head()

,Neighborhood,longitude,latitude,Pivot Cluster Labels
0,West End,-82.997266,42.383644,1.0
1,Chandler Park,-82.974079,42.396291,1.0
2,Riverbend,-82.953467,42.379379,1.0
3,Jefferson Chalmers,-82.945100,42.364698,1.0
4,Riverdale,-83.267268,42.410826,1.0


In [33]:
DV1['Pivot Cluster Labels'].unique()

array([ 1., nan,  2.,  0.,  3.,  4.])

In [34]:
DV1['Pivot Cluster Labels'].fillna(-1, inplace = True) #-1 as a label to indicate that there are no venues in certain neighborhoods
DV1 = DV1.astype({'Pivot Cluster Labels':'int64'})
DV1.dtypes

Neighborhood             object
longitude               float64
latitude                float64
Pivot Cluster Labels      int64
dtype: object

In [35]:
map_clusters_pivot = folium.Map(location=[42.3314, -83.0458], zoom_start=11)


x = np.arange(6)
ys = [i + x + (i*x)**2 for i in range(6)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(DV1['latitude'], DV1['longitude'], DV1['Neighborhood'], DV1['Pivot Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-2],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(map_clusters_pivot)
       
map_clusters_pivot

In [36]:
db_pivot = DBSCAN(eps = 0.0005, min_samples = 2)
DV_pivot.drop(labels = ['Pivot Cluster Labels'], axis = 1, inplace = True)
DV_pivot.set_index('Neighborhood', inplace = True)
db_pivot.fit(DV_pivot)
DV_pivot['DB Pivot Labels'] = db_pivot.labels_
DV_pivot['DB Pivot Labels'].unique()


array([-1,  0,  1,  2,  3,  4])

__After testing several iterations of DBSCAN on the more granular pivot data, it becomes clear that DBSCAN will either assign all the neighborhood to the same cluster or it will produce too many clusters to provide meaningful segmentation__